<a href="https://colab.research.google.com/github/agrawalsourabh/DeepLearning/blob/master/style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMAGE STYLE TRANSFER

**Description**</br>
Content Image + Style Image = Generated Image </br>
Research Paper: [Image Style Transfer Using CNN](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf)
</br>

**Learning Objectives:** </br>

* Deep understanding on Convolution Neural Networks.
* Calculate Loss functions.
* Load the VGG16 model.
* Tuning Hyper Parameters.
* Deploy the model in browser.


In [0]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

import tensorflow as tf
from scipy.optimize import fmin_l_bfgs_b

**Define Hyperparameters**

In [0]:
ITERATIONS = 10
CHANNELS = 3
IMAGE_SIZE = 500
IMAGE_WIDTH = IMAGE_SIZE
IMAGE_HEIGHT = IMAGE_SIZE
IMAGENET_MEAN_RGB_VALUES = [123.68, 116.779, 103.939]
CONTENT_WEIGHT = 0.02
STYLE_WEIGHT = 4.5
TOTAL_VARIATION_WEIGHT = 0.995
TOTAL_VARIATION_LOSS_FACTOR = 1.25


**Image Paths**

In [0]:
input_image_path = "input.png"
style_image_path = "style.png"
output_image_path = "output.png"
combine_image_path = "combined.png"

# Content Image - San Francisco
san_francisco_img_path = "https://www.economist.com/sites/default/files/images/print-edition/20180602_USP001_0.jpg"

# Style Image
tytus_image_path = "http://meetingbenches.com/wp-content/flagallery/tytus-brzozowski-polish-architect-and-watercolorist-a-fairy-tale-in-warsaw/tytus_brzozowski_13.jpg"


**Image Visualisation**

In [0]:
input_image = Image.open(BytesIO(requests.get(san_francisco_img_path).content))
print("Original Image size", input_image.size)

# Resize the image to 500 X 500
input_image = input_image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
print("Original Image size", input_image.size)

input_image.save(input_image_path)

input_image

**Style Visualisation**

In [0]:
style_image = Image.open(BytesIO(requests.get(tytus_image_path).content))
print("Original image size: ", style_image.size)

# Resize image to 500 X 500
style_image = style_image.resize((IMAGE_WIDTH, IMAGE_HEIGHT));
print("After resizing image size: ", style_image.size)

# save style image
style_image.save(style_image_path)

style_image

**Data Normalisation and reshaping to RGB to BGR**

In [0]:
# input image
input_image_array = np.asarray(input_image, dtype = "float32")
print(input_image_array)

input_image_array = np.expand_dims(input_image_array, axis=0)
print("After expand dims: ", input_image_array)

input_image_array[:, :, :, 0] -= IMAGENET_MEAN_RGB_VALUES[2]
input_image_array[:, :, :, 1] -= IMAGENET_MEAN_RGB_VALUES[1]
input_image_array[:, :, :, 2] -= IMAGENET_MEAN_RGB_VALUES[0]
input_image_array = input_image_array[:, :, :, ::-1]

In [0]:
# style_image
style_image_array = np.asarray(style_image, dtype="float32")
style_image_array = np.expand_dims(style_image_array, axis=0)
style_image_array[:, :, :, 0] -= IMAGENET_MEAN_RGB_VALUES[2]
style_image_array[:, :, :, 1] -= IMAGENET_MEAN_RGB_VALUES[1]
style_image_array[:, :, :, 2] -= IMAGENET_MEAN_RGB_VALUES[0]
style_image_array = style_image_array[:, :, :, ::-1]

**Load the model**

In [0]:
# input image
input_image = tf.keras.backend.variable(input_image_array)
input_image

# output image
style_image = tf.keras.backend.variable(style_image_array)
style_image

# combination image
combination_image = tf.keras.backend.placeholder((1, IMAGE_HEIGHT, IMAGE_WIDTH, 3))
combination_image

# concatenate these two images
input_tensor = tf.keras.backend.concatenate([input_image, style_image, combination_image], axis=0)

**Load VGG19 Model** 

In [0]:
model = tf.keras.applications.vgg16.VGG16(input_tensor = input_tensor, include_top=False)

**Model Summary**

In [0]:
model.summary()

In [0]:

def content_loss(content, combination):
    return tf.keras.backend.sum(tf.keras.backend.square(combination - content))

layers = dict([(layer.name, layer.output) for layer in model.layers])

content_layer = "block2_conv2"
layer_features = layers[content_layer]
content_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

loss = tf.keras.backend.variable(0.)
loss = loss+  CONTENT_WEIGHT * content_loss(content_image_features,
                                      combination_features)

In [0]:
def gram_matrix(x):
    features = tf.keras.backend.batch_flatten(tf.keras.backend.permute_dimensions(x, (2, 0, 1)))
    gram = tf.keras.backend.dot(features, tf.keras.backend.transpose(features))
    return gram

In [0]:
def compute_style_loss(style, combination):
    style = gram_matrix(style)
    combination = gram_matrix(combination)
    size = IMAGE_HEIGHT * IMAGE_WIDTH
    return tf.keras.backend.sum(tf.keras.backend.square(style - combination)) / (4. * (CHANNELS ** 2) * (size ** 2))

style_layers = ["block1_conv2", "block2_conv2", "block3_conv3", "block4_conv3", "block5_conv3"]
for layer_name in style_layers:
    layer_features = layers[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    style_loss = compute_style_loss(style_features, combination_features)
    loss = loss + (STYLE_WEIGHT / len(style_layers)) * style_loss

In [0]:
def total_variation_loss(x):
    a = tf.keras.backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, 1:, :IMAGE_WIDTH-1, :])
    b = tf.keras.backend.square(x[:, :IMAGE_HEIGHT-1, :IMAGE_WIDTH-1, :] - x[:, :IMAGE_HEIGHT-1, 1:, :])
    return tf.keras.backend.sum(tf.keras.backend.pow(a + b, TOTAL_VARIATION_LOSS_FACTOR))

loss = loss + TOTAL_VARIATION_WEIGHT * total_variation_loss(combination_image)

In [0]:
outputs = [loss]
outputs = outputs + tf.keras.backend.gradients(loss, combination_image)

def evaluate_loss_and_gradients(x):
    x = x.reshape((1, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    outs = tf.keras.backend.function([combination_image], outputs)([x])
    loss = outs[0]
    gradients = outs[1].flatten().astype("float64")
    return loss, gradients

class Evaluator:

    def loss(self, x):
        loss, gradients = evaluate_loss_and_gradients(x)
        self._gradients = gradients
        return loss

    def gradients(self, x):
        return self._gradients

evaluator = Evaluator()

In [0]:
x = np.random.uniform(0, 255, (1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)) - 128.

for i in range(ITERATIONS):
    x, loss, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.gradients, maxfun=20)
    print("Iteration %d completed with loss %d" % (i, loss))
    
x = x.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
x = x[:, :, ::-1]
x[:, :, 0] = x[:, :, 0] + IMAGENET_MEAN_RGB_VALUES[2]
x[:, :, 1] = x[:, :, 1] + IMAGENET_MEAN_RGB_VALUES[1]
x[:, :, 2] = x[:, :, 2] + IMAGENET_MEAN_RGB_VALUES[0]
x = np.clip(x, 0, 255).astype("uint8")
output_image = Image.fromarray(x)
output_image.save(output_image_path)
output_image